In [1]:
from torchvision import transforms, datasets as ds
import torchvision as tv
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch
import os
import PIL
from PIL import Image
from torchvision import models
'''
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                             ])
data_transforms = {
    'train':transforms.Compose([transforms.Resize((32, 32), PIL.Image.ANTIALIAS),
                                transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ]),
    'val':transforms.Compose([transforms.Resize((32, 32), PIL.Image.ANTIALIAS),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                              ]),
}
'''
data_transforms = {
    'train':transforms.Compose([transforms.Resize((64, 64), PIL.Image.ANTIALIAS),
                                transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ]),
    'val':transforms.Compose([transforms.Resize((64, 64), PIL.Image.ANTIALIAS),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                              ]),
}

In [2]:
data_dir = './10species/'
train_set = {x:ds.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','val']}

#train_set = tv.datasets.ImageFolder(root='./data_plants', transform=transform)

In [3]:
trainloader = {x: torch.utils.data.DataLoader(train_set[x],     
                                            batch_size=4, 
                                            shuffle=True,
                                            num_workers=2)
                                            for x in ['train','val']}
#trainloader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True, num_workers=2)

In [4]:
print(len(trainloader['train']))

1630


In [5]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [6]:
classes = ('43', '105', '78', '96',
           '46', '49', '127', '54', '66', '85')

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) 
        self.pool  = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(400, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
        self.drop  = nn.Dropout(p=0.5)
 
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 400)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.drop(x)
        return x

In [11]:
class zyz_Net(nn.Module):
    def __init__(self):
        super(zyz_Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 9) 
        self.pool  = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 9)
        self.fc1   = nn.Linear(1600, 200)
        self.fc2   = nn.Linear(200, 84)
        self.fc3   = nn.Linear(84, 10)
        self.drop  = nn.Dropout(p=0.5)
 
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 1600)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.drop(x)
        return x

In [12]:
net = zyz_Net()
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim 
criterion = nn.CrossEntropyLoss() 
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr = 0.001)

In [20]:
#for epoch in range(2): 
is_mixup = False
use_gpu = False#True
i = 0     
running_loss = 0.0
epoches = 3
for epoch in range(epoches):
    for data in trainloader['train']: 
        inputs, labels = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            net.cuda()
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        if is_mixup:
            inputs, targets_a, targets_b, lam = mixup_data(inputs, labels, alpha=1.0, use_cuda=False)
            inputs, targets_a, targets_b = map(Variable, (inputs,targets_a, targets_b))
            outputs = net(inputs)
            optimizer.zero_grad()
            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
            loss.backward()
            optimizer.step()   
        else:
            optimizer.zero_grad() 
            outputs = net(inputs)
        #print(outputs.shape)
            loss = criterion(outputs, labels) 
            loss.backward() 
            optimizer.step() 
        i += 1 
        running_loss += loss.item()  
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (i-99, i+1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[0,   100] loss: 1.471
[100,   200] loss: 1.563
[200,   300] loss: 1.454
[300,   400] loss: 1.495
[400,   500] loss: 1.485
[500,   600] loss: 1.415
[600,   700] loss: 1.495
[700,   800] loss: 1.479
[800,   900] loss: 1.461
[900,  1000] loss: 1.573
[1000,  1100] loss: 1.504
[1100,  1200] loss: 1.478
[1200,  1300] loss: 1.473
[1300,  1400] loss: 1.481
[1400,  1500] loss: 1.471
[1500,  1600] loss: 1.542
[1600,  1700] loss: 1.330
[1700,  1800] loss: 1.415
[1800,  1900] loss: 1.372
[1900,  2000] loss: 1.476
[2000,  2100] loss: 1.428
[2100,  2200] loss: 1.441
[2200,  2300] loss: 1.564
[2300,  2400] loss: 1.247
[2400,  2500] loss: 1.386
[2500,  2600] loss: 1.492
[2600,  2700] loss: 1.366
[2700,  2800] loss: 1.510
[2800,  2900] loss: 1.452
[2900,  3000] loss: 1.493
[3000,  3100] loss: 1.505
[3100,  3200] loss: 1.407
[3200,  3300] loss: 1.506
[3300,  3400] loss: 1.451
[3400,  3500] loss: 1.393
[3500,  3600] loss: 1.380
[3600,  3700] loss: 1.365
[3700,  3800] loss: 1.358
[3800,  3900] loss: 1.37

In [17]:
correct = 0
total = 0
running_loss = 0.0
i = 0
for data in trainloader['val']:
    images, labels = data
    images.cpu()
    labels.cpu()
    net.cpu()
    outputs = net(Variable(images))
    loss = criterion(outputs, labels) 
    running_loss += loss.item()  
    i += 1
    if i % 60 == 59:
        print('[%d, %5d] loss: %.3f' % (i-59, i+1, running_loss / 60))
        running_loss = 0.0
    _, predicted = torch.max(outputs.data, 1) 
    total += 4
    correct += (predicted == labels).sum()   

print('Accuracy: %d %%' % (100 * correct / total))

[0,    60] loss: 2.004
[60,   120] loss: 2.023
[120,   180] loss: 2.040
[180,   240] loss: 2.005
[240,   300] loss: 2.026
[300,   360] loss: 1.897
Accuracy: 29 %


In [64]:
state = {'net':net.state_dict(), 'optimizer':optimizer.state_dict()}
torch.save(state,'haha10.pth')

##  

## 咸鱼版本

In [65]:
import torch.nn as nn
import torch.nn.functional as F

'''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(400, 200)
        self.fc2 = nn.Linear(200, 160)
        self.fc3 = nn.Linear(160, 102)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 400)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
'''


class Net2(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 96, 11, 4, 0),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(3,2)
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(96, 256, 5, 1, 2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(3,2)
        )
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv2d(256,384, 3, 1, 1),
            torch.nn.ReLU(),
        )
        self.conv4 = torch.nn.Sequential(
            torch.nn.Conv2d(384,384, 3, 1, 1),
            torch.nn.ReLU(),
        )
        self.conv5 = torch.nn.Sequential(
            torch.nn.Conv2d(384,256, 3, 1, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(3,2)
        )
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(9216, 4096),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 10)
        )
 
    def forward(self, x):
        conv1_out = self.conv1(x)
        conv2_out = self.conv2(conv1_out)
        conv3_out = self.conv3(conv2_out)
        conv4_out = self.conv4(conv3_out)
        conv5_out = self.conv5(conv4_out)
        res = conv5_out.view(conv5_out.size(0), -1)
        out = self.dense(res)
        return out

    '''def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 96, 11, stride=4)
        self.pool1 = nn.MaxPool2d(3, 2)
        self.conv2 = nn.Conv2d(96, 256, 5, padding=2)
        self.pool2 = nn.MaxPool2d(3, 2)
        self.conv3 = nn.Conv2d(256, 384, 3, padding=1)
        self.conv4 = nn.Conv2d(384, 384, 3, padding=1)
        self.conv5 = nn.Conv2d(384, 256, 3, padding=1)
        self.pool5 = nn.MaxPool2d(3, 2)
        self.fc1 = nn.Linear(9216, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 102)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool5(F.relu(self.conv5(x)))
        x = x.view(-1, 9216)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x'''

In [66]:
from torchvision import transforms, datasets as ds
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch
import os
import PIL
from PIL import Image
import torch.optim as optim

data_transforms = {
    'train':transforms.Compose([transforms.Resize((227, 227), PIL.Image.ANTIALIAS),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                ]),
    'val':transforms.Compose([transforms.Resize((227, 227), PIL.Image.ANTIALIAS),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                              ]),
}


data_dir = './10species/'
train_set = {x:ds.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','val']}


trainloader = {x: torch.utils.data.DataLoader(train_set[x],
                                              batch_size=4,
                                              shuffle=True,
                                              num_workers=2)
                                    for x in ['train','val']}

net = Net2()


TypeError: super(type, obj): obj must be an instance or subtype of type

In [67]:
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(net.parameters(), lr = 0.001)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

i = 0
running_loss = 0.0

use_gpu = True

TRAINING_STEPS = 3

for op in range(TRAINING_STEPS):
    print('In loop %d' % op)
    for data in trainloader['train']:
        inputs, labels = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            net.cuda()
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        i += 1

        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (i - 99, i + 1, loss))


print('Finished Training')


correct = 0
total = 0
running_loss = 0.0
i = 0


In loop 0


RuntimeError: invalid argument 2: size '[-1 x 400]' is invalid for input with 179776 elements at ..\aten\src\TH\THStorage.cpp:80

In [8]:
for data in trainloader['val']:
    images, labels = data
    images.cpu()
    labels.cpu()
    net.cpu()
    outputs = net(Variable(images))
    loss = criterion(outputs, labels)
    i += 1
    if i % 60 == 59:
        print('[%d, %5d] loss: %.3f' % (i-59, i+1, loss))
    _, predicted = torch.max(outputs.data, 1)
    total += 4
    correct += (predicted == labels).sum()

print('Accuracy: %d %%' % (100 * correct / total))

[0,    60] loss: 4.884
[60,   120] loss: 4.935
Accuracy: 0 %
